In [22]:
import numpy as np
import pdb
from pandas_plink import read_plink
from limix_lmm import LMM
import pylab as plt
import os
import pandas as pd

## Loading Genetic Data from 1000 Genomes

In this code segment, we are dealing with genetic data derived from the 1000 Genomes Project. Let's break down some context and terminology:

- **1000 Genomes Dataset**: 
  - A comprehensive public resource that provides data on the genetic makeup of 2504 individuals from 26 different populations around the world.
  - It's an effort to sequence the genomes of a large number of people to provide a comprehensive resource on human genetic variation.
  - In our current scenario, we are utilizing just a subset of this extensive dataset.

- **PLINK Files**:
  - PLINK is a free, open-source whole genome association analysis toolset, which operates on a range of binary dataset types.
  - The dataset is typically represented in three primary files:
    - `.bim` (Binary Marker File): Contains information about the genetic markers (like SNPs).
    - `.fam` (Family File): Contains information about the individuals (like their IDs, phenotype).
    - `.bed` (Binary PED File): Contains the genotype data in a binary format.
  - Together, these files give a comprehensive picture of both the individuals in the dataset and their genetic variations.

- **Loading Data in Python**:
  - The `read_plink` function from the `pandas_plink` library in Python can be used to load PLINK binary datasets.
  - Once loaded, we can manipulate and analyze this data using standard Python tools and libraries.

The code `bim, fam, G = read_plink(bfile)` essentially reads the genetic data from the specified PLINK binary files and stores the marker information in `bim`, the family/individual information in `fam`, and the genotype matrix in `G`.


In [2]:
# load genetic data
bfile = '../../data/ALL.chr22_GRCh38.genotypes.20170504/ALL.chr22_GRCh38.genotypes.20170504'
bim, fam, G = read_plink(bfile)

Mapping files: 100%|██████████████████████████████| 3/3 [00:00<00:00, 16.97it/s]


In [3]:
def extract_region(bim, G, chrom, start, end):
    I1 = bim['chrom']==str(chrom)
    I2 = bim['pos'] > start
    I3 = bim['pos'] < end
    Ikeep = I1 * I2 * I3
    bim = bim.loc[Ikeep].copy()
    G = G[bim['i'].values]
    bim['i'] = np.arange(len(bim))
    return bim, G

In [4]:
def subset_individuals(fam, G, keep_idxs):
    fam = fam.iloc[keep_idxs]
    G = G[:, keep_idxs]
    return bim, G

In [5]:
# subset to 1000 individuals
N = 1000
np.random.seed(0)
idxs = np.sort(np.random.permutation(fam.shape[0])[:N])
fams, Gs = subset_individuals(fam, G, idxs)

In [6]:
# extract region
chrom = 22
start = 30797531
size = 100000
bimr, Gr = extract_region(bim, Gs, chrom, start, start + size)

In [7]:
# load genotype matrix
X_real = Gr.compute().T